In [14]:
import matplotlib 
import pandas as pd
import pickle
import numpy as np
import scipy as sp
import math
import re
import pymysql as db
import os
import sklearn
import datetime
from matplotlib import pyplot as plt
from sshtunnel import SSHTunnelForwarder
from sklearn import model_selection, svm
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import classification_report, average_precision_score, mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import label_binarize
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import r2_score
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_curve, roc_auc_score

from mlxtend.plotting import plot_decision_regions
import matplotlib.gridspec as gridspec
import itertools
import time

from pyglmnet import GLM
from catboost import CatBoostRegressor
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
from mlxtend.classifier import StackingClassifier

from scipy.io import loadmat
import scipy


#For Seaborn
import seaborn as sns
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 30,10



#For Arima
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA

#For NLP
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import re
import string
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer

#For Bayesian Optimization
#import bayes_opt
#from bayes_opt import BayesianOptimization


#Dask Framework
import dask.dataframe as dd

#For View
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\Student/nltk_data'
    - 'C:\\Users\\Student\\anaconda3\\nltk_data'
    - 'C:\\Users\\Student\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\Student\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Student\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
#Data Ingestion
fitbit_data=pd.read_csv(r"Subject_1_Fitbit.csv",sep=',',encoding="utf-8",error_bad_lines=False,low_memory = False)
wavelet_data=pd.read_csv(r"Subject_1_Wavelet.csv",sep=',',encoding="utf-8",error_bad_lines=False,low_memory = False)
print(fitbit_data.shape)
print(wavelet_data.shape)

(397, 3)
(23814, 10)


In [21]:
#sklear data ingestion
import pandas as pd

fitbit_data=pd.read_csv(r"Subject_1_Fitbit.csv",sep=',',encoding="utf-8",error_bad_lines=False,low_memory = False)
wavelet_data=pd.read_csv(r"Subject_1_Wavelet.csv",sep=',',encoding="utf-8",error_bad_lines=False,low_memory = False)
print(fitbit_data.shape)
print(wavelet_data.shape)

(397, 3)
(23814, 10)


In [22]:
fitbit_data.head()

Time  State Interpreted
0  05:46      2       Awake
1  05:47      2       Awake
2  05:48      2       Awake
3  05:49      2       Awake
4  05:50      2       Awake

In [23]:
wavelet_data.head()

ts           tsUnixNano   accel_x   accel_y   accel_z  \
0  2018-11-06T20:46:00Z  1541537160000000000  8.571167  3.165926  0.077218   
1  2018-11-06T20:46:01Z  1541537161000000000  8.802820  3.243144  0.463306   
2  2018-11-06T20:46:02Z  1541537162000000000  8.493949  3.243144  1.081048   
3  2018-11-06T20:46:03Z  1541537163000000000  8.493949  3.243144  1.081048   
4  2018-11-06T20:46:04Z  1541537164000000000  8.493949  3.243144  0.926613   

   gyro_x  gyro_y  gyro_z timestamp_secs timestamp_mins  
0     NaN     NaN     NaN       05:46:00          05:46  
1     NaN     NaN     NaN       05:46:01          05:46  
2     NaN     NaN     NaN       05:46:02          05:46  
3     NaN     NaN     NaN       05:46:03          05:46  
4     NaN     NaN     NaN       05:46:04          05:46

In [27]:
import datetime
def rev_egg(date_time):
    date_time=date_time/1000000000 #Nanoseconds to Seconds
    x=datetime.datetime.fromtimestamp(date_time).strftime('%H:%M:%S')
    return x

wavelet_data['timestamp_secs']=wavelet_data['tsUnixNano'].apply(rev_egg)

In [28]:
import datetime
def rev_egg_min(date_time):
    date_time=date_time/1000000000 #Nanoseconds to Seconds
    x=datetime.datetime.fromtimestamp(date_time).strftime('%H:%M')
    return x

wavelet_data['timestamp_mins']=wavelet_data['tsUnixNano'].apply(rev_egg_min)

In [29]:
wavelet_data.head()

ts           tsUnixNano   accel_x   accel_y   accel_z  \
0  2018-11-06T20:46:00Z  1541537160000000000  8.571167  3.165926  0.077218   
1  2018-11-06T20:46:01Z  1541537161000000000  8.802820  3.243144  0.463306   
2  2018-11-06T20:46:02Z  1541537162000000000  8.493949  3.243144  1.081048   
3  2018-11-06T20:46:03Z  1541537163000000000  8.493949  3.243144  1.081048   
4  2018-11-06T20:46:04Z  1541537164000000000  8.493949  3.243144  0.926613   

   gyro_x  gyro_y  gyro_z timestamp_secs timestamp_mins  
0     NaN     NaN     NaN       05:46:00          05:46  
1     NaN     NaN     NaN       05:46:01          05:46  
2     NaN     NaN     NaN       05:46:02          05:46  
3     NaN     NaN     NaN       05:46:03          05:46  
4     NaN     NaN     NaN       05:46:04          05:46

In [ ]:
fitbit_start=

In [ ]:
print(fitbit_start)
print(fitbit_end)

In [20]:
#refugeeswavelet_data.head(10)